# Create pickle file of declustered catalogue

## Importing libraries

In [1]:
%matplotlib inline
import os
import re
import sys
import h5py
import numpy
import subprocess
try:
    import cPickle as pickle
except:
    import pickle
from copy import deepcopy
from oqmbt.oqt_project import OQtProject, OQtModel, OQtSource
from openquake.hmtk.parsers.catalogue.csv_catalogue_parser import CsvCatalogueParser
from openquake.hmtk.seismicity.selector import CatalogueSelector

In [2]:
#
# load project information 
project_pickle_filename = os.environ.get('OQMBT_PROJECT')
oqtkp = OQtProject.load_from_file(project_pickle_filename)
oqtkp.directory = os.path.dirname(project_pickle_filename)
model_id = oqtkp.active_model_id
model = oqtkp.models[model_id]
#
# set the full path of the catalogue
catalogue_csv_filename = os.path.abspath(os.path.join(oqtkp.directory, getattr(model,'catalogue_csv_filename')))
#
# printing info
print ('Model id ...: {:s}'.format(model_id))
print ('Reading ....: {:s}'.format(catalogue_csv_filename))

Model id ...: model01
Reading ....: /Users/mpagani/Repos/model_building_tools/oqmbt/tests/notebooks/data/catalogue.csv


## Reading information from the project
 * The earthquake catalogue is an hmtk-formatted catalogue. A common file format used to represent this typology of infomation is an ASCII comma-separated file (.csv)

## Main settings

In [3]:
#
# cutoff magnitude is the lowest magnitude treshold. All the events below this threshold 
# are filtered out at the beginning of the processing workflow
cutoff_magnitude = float(model.catalogue_cutoff_magnitude)
#
# width of bins used to discretise the MFDs
binwidth = float(model.mfd_binwidth)

## Read catalogue
Read the catalogue from the .csv file and remove events below the cutoff magnitude.

In [4]:
catalogue_parser = CsvCatalogueParser(catalogue_csv_filename)
catalogue = catalogue_parser.read_file()
#
# filtering catalogue
selector = CatalogueSelector(catalogue, create_copy=False)
selector.within_magnitude_range(cutoff_magnitude, 10.)
dep_max = float(model.catalogue_maximum_depth)
dep_min = float(model.catalogue_minimum_depth)
tmp = selector.within_depth_range(dep_max, dep_min)
#
# info
print ('Catalogue:')
print ('    minimum hypocentral depth: {:.2f}'.format(numpy.min(catalogue.data['depth'])))
print ('    maximum hypocentral depth: {:.2f}'.format(numpy.max(catalogue.data['depth'])))

Catalogue Attribute LonError is not a recognised catalogue key
Catalogue Attribute MagType is not a recognised catalogue key
Catalogue Attribute LocId is not a recognised catalogue key
Catalogue Attribute MagError is not a recognised catalogue key
Catalogue Attribute LatError is not a recognised catalogue key
Catalogue Attribute LocCode is not a recognised catalogue key
Catalogue Attribute SecError is not a recognised catalogue key
Catalogue Attribute DepError is not a recognised catalogue key
Catalogue Attribute MagCode is not a recognised catalogue key
Catalogue Attribute Log is not a recognised catalogue key
Catalogue Attribute MagId is not a recognised catalogue key


TypeError: unorderable types: numpy.ndarray() >= str()

## Saving catalogue pickle file and updating the project 

In [ ]:
model_lab = re.sub('\s', '_', oqtkp.models[model_id].name).lower()
output_path = os.path.join(oqtkp.directory, '%s_catalogue.pkl' % model_lab)
fou = open(output_path,'wb') 
pickle.dump(catalogue, fou)
fou.close()
print ('Catalogue dataset saved into file: \n%s' % (output_path))

In [ ]:
oqtkp.models[model_id].declustered_catalogue_pickle_filename = os.path.relpath(output_path, oqtkp.directory)
oqtkp.save()
print (oqtkp.models[model_id].declustered_catalogue_pickle_filename)